In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
cp = '/content/Customers.csv'
pp = '/content/Products.csv'
tp = '/content/Transactions.csv'

In [4]:
customers = pd.read_csv(cp)
products = pd.read_csv(pp)
transactions = pd.read_csv(tp)

In [5]:
customers.head()


,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [6]:
transactions.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [7]:
products.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [8]:
customers_info = customers.info()
products_info = products.info()
transactions_info = transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  


In [9]:

customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

# mergig  datasets for analysis
# left join because we want to focus on transcation
transactions_customers = transactions.merge(customers, on='CustomerID', how='left')


merged_data = transactions_customers.merge(products, on='ProductID', how='left')
merged_data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [18]:

merged_df = merged_data

customer_features = merged_df.groupby("CustomerID").agg(
    TotalTransactions=("TransactionID", "count"),
    TotalQuantity=("Quantity", "sum"),
    TotalSpend=("TotalValue", "sum"),
    AvgSpendPerTransaction=("TotalValue", "mean"),
    DistinctCategories=("Category", "nunique"),
).reset_index()


customer_features = customer_features.merge(customers, on="CustomerID", how="left")


merged_df['TransactionDate'] = pd.to_datetime(merged_df['TransactionDate'])
latest_transaction_date = merged_df['TransactionDate'].max()

customer_features['SignupRecency'] = (
    latest_transaction_date - pd.to_datetime(customer_features['SignupDate'])
).dt.days


customer_features = customer_features.drop(columns=["SignupDate"])


customer_features = pd.get_dummies(customer_features, columns=["Region"], drop_first=True)


display(customer_features)


,CustomerID,TotalTransactions,TotalQuantity,TotalSpend,AvgSpendPerTransaction,DistinctCategories,CustomerName,SignupRecency,Region_Europe,Region_North America,Region_South America
0,C0001,5,12,3354.52,670.904000,3,Lawrence Carroll,902,False,False,True
1,C0002,4,10,1862.74,465.685000,2,Elizabeth Lutz,1049,False,False,False
2,C0003,4,14,2725.38,681.345000,3,Michael Rivera,296,False,False,True
3,C0004,8,23,5354.88,669.360000,3,Kathleen Rodriguez,811,False,False,True
4,C0005,3,7,2034.24,678.080000,2,Laura Weber,866,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,4,12,4982.88,1245.720000,3,Laura Watts,935,True,False,False
195,C0197,3,9,1928.65,642.883333,2,Christina Harvey,648,True,False,False
196,C0198,2,3,931.83,465.915000,2,Rebecca Ray,1035,True,False,False
197,C0199,4,9,1979.28,494.820000,2,Andrea Jenkins,756,True,False,False


In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

numerical_cols = [
    "TotalTransactions", "TotalQuantity", "TotalSpend",
    "AvgSpendPerTransaction", "DistinctCategories", "SignupRecency"
]

scaler = StandardScaler()
customer_features_scaled = scaler.fit_transform(customer_features[numerical_cols])

region_cols = [col for col in customer_features.columns if col.startswith("Region_")]
all_features_scaled = np.hstack([customer_features_scaled, customer_features[region_cols].values])


similarity_matrix = cosine_similarity(all_features_scaled)

top_20_customers = customer_features['CustomerID'][:20]
lookalike_results = {}

for i, cust_id in enumerate(top_20_customers):
    scores = list(enumerate(similarity_matrix[i]))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:4]

    lookalike_results[cust_id] = [
        {"cust_id": customer_features.loc[j, "CustomerID"], "score": round(score, 4)}
        for j, score in sorted_scores
    ]

lookalike_df = pd.DataFrame([
    {"cust_id": cust_id, "lookalikes": lookalikes}
    for cust_id, lookalikes in lookalike_results.items()
])

lookalike_path = '/content/Lookalike.csv'
lookalike_df.to_csv(lookalike_path, index=False)

lookalike_path


'/content/Lookalike.csv'